In [84]:
import numpy as np
from itertools import product
import pandas as pd
import foraging_toolkit as ft
import environments
import simulation
import simulation_old 
import success_metrics
import imp
imp.reload(ft)
imp.reload(simulation)
imp.reload(simulation_old)
imp.reload(environments)
imp.reload(success_metrics)


<module 'success_metrics' from 'c:\\Users\\admin\\Dropbox\\Code\\Basis-code\\new-collaborative-intelligence\\mx_refactored\\success_metrics.py'>

Indpendent variables in the forward model. Specify the parameter values for all the cases you are interested in. 

In [85]:
# Parameters of interest in our communication model 
c_trust = np.array([0, 0.5, 1])
sight_radius = [5]
reward_patch_dim = [1, 4]

# List of all possible combination of parameter values 
param_list = [i for i in product(c_trust, sight_radius, reward_patch_dim)]

print(len(param_list))
print('param_list', param_list)




6
param_list [(0.0, 5, 1), (0.0, 5, 4), (0.5, 5, 1), (0.5, 5, 4), (1.0, 5, 1), (1.0, 5, 4)]


Generate a dataframe containing all possible combinations of the parameter values specified above.

In [86]:
#each row of both dataframes corresponds to one experiment.  
#columns are parameters 
metadataDF = pd.DataFrame(param_list)

# success_metricsDF = pd.DataFrame(results)
# allresultsDF = pd.concat([metadataDF, success_metricsDF], axis='columns') # concatenate metadata and results 
metadataDF.columns = ["c_trust", "sight_radius", "reward_patch_dim"] 
metadataDF.head()
display(metadataDF)

,c_trust,sight_radius,reward_patch_dim
0,0.0,5,1
1,0.0,5,4
2,0.5,5,1
3,0.5,5,4
4,1.0,5,1
5,1.0,5,4


*Part 2*
Test run one of the simulations from metadata (one row)

In [87]:
N_frames = 20
N_agents = 2

nsims = len(metadataDF)

nsims = 1 # TEST
for si in range(nsims):
    # 1. pull out parameters from row si in the metadata

    # 2. Feed the parameters through the run() function once (later we'll do this many times)
    # Create an initialize a new environment 
    env = environments.Environment(edge_size=30, N_total_food_units=4, patch_dim=2, max_step_size=3)
    env.add_food_patches(food_statistics_type="drop_food_once")
    sim = simulation_old.Simulation(env, N_agents=N_agents, N_frames=N_frames)
    sim.run()
    # print(sim.all_birdsDF)
    # print(sim.all_rewardsDF) 
    # by this point we should have birdlocsDF and rewardlocsDF 

    # # 3. Compute success measures 

    # time_to_first_allbirds = np.zeros(N_agents)

    # for bird_id in range(1, N_agents + 1):
    #     singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == bird_id]  
    #     display(singlebirdDF)
    #     time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(singlebirdDF, sim.all_rewardsDF, N_frames)

    #     print(time_to_first_allbirds)

    # num_birds_failed = np.sum(time_to_first_allbirds < 0)
    # print(num_birds_failed)

    # #4. Combine the metadata and the success measures into the results dataframe 

    # 5. Save the raw data into a separate folder.


    
    
# Save the results dataframe into the same folder as the metadata


Use the dataframes to generate animations

In [88]:
communicators = ft.object_from_data(sim.all_birdsDF, sim.all_rewardsDF)
ft.animate_birds(communicators, plot_rewards=True,
                  width = 600, height = 600, point_size = 10)

In [101]:
print(sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == 2])

       x     y  time  bird    type
0   17.0  16.0     1     2  random
1   16.0  16.0     2     2  random
2   17.0  14.0     3     2  random
3   17.0  13.0     4     2  random
4   16.0  12.0     5     2  random
5   17.0  10.0     6     2  random
6   15.0   8.0     7     2  random
7   13.0   6.0     8     2  random
8   11.0   4.0     9     2  random
9   10.0   2.0    10     2  random
10  10.0   2.0    11     2  random
11  11.0   2.0    12     2  random
12  10.0   2.0    13     2  random
13  11.0   2.0    14     2  random
14  11.0   2.0    15     2  random
15  10.0   2.0    16     2  random
16  10.0   2.0    17     2  random
17  10.0   2.0    18     2  random
18  11.0   2.0    19     2  random
19   0.0   0.0    20     2  random


In [100]:
# print(sim.all_rewardsDF[]) 
print(sim.all_rewardsDF.loc[sim.all_rewardsDF['time'] == 4])


    x  y  time
0  10  1     4
1  11  1     4
2  10  2     4
3  11  2     4


In [125]:
imp.reload(success_metrics)
time_to_first_allbirds = np.zeros(N_agents)
print(time_to_first_allbirds.shape)

for bird_id in range(1, N_agents + 1):
    # bird_id = 2
    singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == bird_id] ## TO DO: change this to a boolean operation checking if the values satisfy a constraint
    display(singlebirdDF)
    time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(singlebirdDF, sim.all_rewardsDF, N_frames)

    print(time_to_first_allbirds)

num_birds_failed = np.sum(time_to_first_allbirds < 0)
print(num_birds_failed)


(2,)


,x,y,time,bird,type
0,19.0,17.0,1,1,random
1,22.0,17.0,2,1,random
2,20.0,16.0,3,1,random
3,22.0,17.0,4,1,random
4,23.0,18.0,5,1,random
5,22.0,16.0,6,1,random
6,23.0,17.0,7,1,random
7,24.0,17.0,8,1,random
8,24.0,18.0,9,1,random
9,25.0,17.0,10,1,random


[-1.  0.]


,x,y,time,bird,type
0,17.0,16.0,1,2,random
1,16.0,16.0,2,2,random
2,17.0,14.0,3,2,random
3,17.0,13.0,4,2,random
4,16.0,12.0,5,2,random
5,17.0,10.0,6,2,random
6,15.0,8.0,7,2,random
7,13.0,6.0,8,2,random
8,11.0,4.0,9,2,random
9,10.0,2.0,10,2,random


[-1.  9.]
1
